# Лабораторная работа №2. Градиентный бустинг

В рамках данной лабораторной работы предстоит реализовать алгоритм градиентного бустинга и сравнить его с эталонной реализацией из библиотеки `scikit-learn`.

## Задание

1. Выбрать датасет для анализа, например, на [kaggle](https://www.kaggle.com/datasets).
2. Реализовать алгоритм градиентного бустинга.
3. Обучить модель на выбранном датасете.
4. Оценить качество модели с использованием кросс-валидации.
5. Замерить время обучения модели.
6. Сравнить результаты с эталонной реализацией из библиотеки [scikit-learn](https://scikit-learn.org/stable/):
   * точность модели;
   * время обучения.
7. Подготовить отчет, включающий:
   * описание алгоритма градиентного бустинга;
   * описание датасета;
   * результаты экспериментов;
   * сравнение с эталонной реализацией;
   * выводы.

# 1. Выбрать датасет для анализа
### В качестве датасета был выбран набор данных для предсказания цены дома
### Таргет: price

### Ссылка:

https://www.kaggle.com/datasets/denkuznetz/housing-prices-regression

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("denkuznetz/housing-prices-regression")

print("Path to dataset files:", path)

c:\Users\klimm\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 23.6k/23.6k [00:00<00:00, 17.0MB/s]

Extracting files...
Path to dataset files: C:\Users\klimm\.cache\kagglehub\datasets\denkuznetz\housing-prices-regression\versions\1


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('real_estate_dataset.csv')

In [3]:
df.shape

(500, 12)

In [4]:
df.head()

,ID,Square_Feet,Num_Bedrooms,Num_Bathrooms,Num_Floors,Year_Built,Has_Garden,Has_Pool,Garage_Size,Location_Score,Distance_to_Center,Price
0,1,143.635030,1,3,3,1967,1,1,48,8.297631,5.935734,602134.816747
1,2,287.678577,1,2,1,1949,0,1,37,6.061466,10.827392,591425.135386
2,3,232.998485,1,3,2,1923,1,0,14,2.911442,6.904599,464478.696880
3,4,199.664621,5,2,2,1918,0,0,17,2.070949,8.284019,583105.655996
4,5,89.004660,4,3,3,1999,1,0,34,1.523278,14.648277,619879.142523


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  500 non-null    int64  
 1   Square_Feet         500 non-null    float64
 2   Num_Bedrooms        500 non-null    int64  
 3   Num_Bathrooms       500 non-null    int64  
 4   Num_Floors          500 non-null    int64  
 5   Year_Built          500 non-null    int64  
 6   Has_Garden          500 non-null    int64  
 7   Has_Pool            500 non-null    int64  
 8   Garage_Size         500 non-null    int64  
 9   Location_Score      500 non-null    float64
 10  Distance_to_Center  500 non-null    float64
 11  Price               500 non-null    float64
dtypes: float64(4), int64(8)
memory usage: 47.0 KB


In [6]:
df.describe()

,ID,Square_Feet,Num_Bedrooms,Num_Bathrooms,Num_Floors,Year_Built,Has_Garden,Has_Pool,Garage_Size,Location_Score,Distance_to_Center,Price
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,250.500000,174.640428,2.958000,1.976000,1.964000,1957.604000,0.536000,0.492000,30.174000,5.164410,10.469641,582209.629529
std,144.481833,74.672102,1.440968,0.820225,0.802491,35.491781,0.499202,0.500437,11.582575,2.853489,5.588197,122273.390345
min,1.000000,51.265396,1.000000,1.000000,1.000000,1900.000000,0.000000,0.000000,10.000000,0.004428,0.062818,276892.470136
25%,125.750000,110.319923,2.000000,1.000000,1.000000,1926.000000,0.000000,0.000000,20.000000,2.760650,6.066754,503080.344140
50%,250.500000,178.290937,3.000000,2.000000,2.000000,1959.000000,1.000000,0.000000,30.000000,5.206518,10.886066,574724.113347
75%,375.250000,239.031220,4.000000,3.000000,3.000000,1988.000000,1.000000,1.000000,41.000000,7.732933,15.072590,665942.301274
max,500.000000,298.241199,5.000000,3.000000,3.000000,2022.000000,1.000000,1.000000,49.000000,9.995439,19.927966,960678.274291


Нет пропусков

Удалим ID

In [10]:
X = df.drop(['ID', 'Price'], axis=1)
y = df['Price']

# 2. Реализовать алгоритм градиентного бустинга.

In [14]:
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor

In [16]:
class CustomGradientBoosting:
    """
    Реализация градиентного бустинга для регрессии
    """
    
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3, random_state=None):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.random_state = random_state
        
        # Списки для хранения моделей и их весов
        self.models = []
        self.train_errors = []
        self.feature_importances_ = None
        
    def _calculate_residuals(self, y_true, y_pred):
        """Вычисление остатков (градиентов для MSE)"""
        return y_true - y_pred
    
    def _initial_prediction(self, y):
        """Начальное предсказание (среднее значение для MSE)"""
        return np.mean(y)
    
    def fit(self, X, y):
        """Обучение модели градиентного бустинга"""
        # Преобразование в numpy arrays
        X = np.array(X)
        y = np.array(y)
        
        # Инициализация
        self.models = []
        self.train_errors = []
        
        # Начальное предсказание (константа)
        self.initial_prediction = self._initial_prediction(y)
        current_predictions = np.full(len(y), self.initial_prediction)
        
        # Инициализация важности признаков
        self.feature_importances_ = np.zeros(X.shape[1])
        
        # Итеративное добавление моделей
        for i in range(self.n_estimators):
            # Вычисление остатков (псевдо-остатков)
            residuals = self._calculate_residuals(y, current_predictions)
            
            # Обучение нового слабого ученика на остатках
            tree = DecisionTreeRegressor(
                max_depth=self.max_depth,
                random_state=self.random_state + i if self.random_state else None
            )
            tree.fit(X, residuals)
            
            # Предсказание нового дерева
            tree_predictions = tree.predict(X)
            
            # Обновление общих предсказаний с learning rate
            current_predictions += self.learning_rate * tree_predictions
            
            # Сохранение модели
            self.models.append(tree)
            
            # Накопление важности признаков
            self.feature_importances_ += tree.feature_importances_
            
            # Вычисление ошибки на обучающей выборке (используем импорт из sklearn)
            train_error = mean_squared_error(y, current_predictions)
            self.train_errors.append(train_error)
        
        # Нормализация важности признаков
        self.feature_importances_ /= self.n_estimators
        
        return self
    
    def predict(self, X):
        """Предсказание для новых данных"""
        X = np.array(X)
        
        # Начинаем с начального предсказания
        predictions = np.full(len(X), self.initial_prediction)
        
        # Добавляем предсказания каждого дерева
        for tree in self.models:
            tree_predictions = tree.predict(X)
            predictions += self.learning_rate * tree_predictions
            
        return predictions

# 3-5. Обучить модель на выбранном датасете, оценить качество модели с использованием кросс-валидации и замерить время обучения модели.

In [6]:
import time
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [11]:
# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
print("Кастомный градиентный бустинг")

# Кросс-валидация
cv_folds = KFold(n_splits=5, shuffle=True, random_state=42)

def custom_cross_validation(model_class, X, y, cv_folds, **model_params):
    fold_scores_r2 = []
    fold_times = []
    
    for fold, (train_idx, val_idx) in enumerate(cv_folds.split(X), 1):
        print(f"Фолд {fold}/5...", end=" ")
        
        X_fold_train, X_fold_val = X.iloc[train_idx], X.iloc[val_idx]
        y_fold_train, y_fold_val = y.iloc[train_idx], y.iloc[val_idx]
        
        fold_model = model_class(**model_params)
        
        fold_start_time = time.time()
        fold_model.fit(X_fold_train, y_fold_train)
        fold_pred = fold_model.predict(X_fold_val)
        fold_time = time.time() - fold_start_time
        
        fold_r2 = r2_score(y_fold_val, fold_pred)
        
        fold_scores_r2.append(fold_r2)
        fold_times.append(fold_time)
        
        print(f"R²: {fold_r2:.4f}, Время: {fold_time:.2f}с")
    
    return fold_scores_r2, fold_times

# Проведение кросс-валидации
cv_start_time = time.time()
cv_r2_scores, cv_times = custom_cross_validation(
    CustomGradientBoosting, X_train, y_train, cv_folds,
    n_estimators=50, learning_rate=0.1, max_depth=4, random_state=42
)
total_cv_time = time.time() - cv_start_time

# Результаты
print(f"\nКросс-валидация:")
print(f"R²: {np.mean(cv_r2_scores):.4f} ± {np.std(cv_r2_scores):.4f}")
print(f"Время: {np.mean(cv_times):.2f} ± {np.std(cv_times):.2f} секунд")

Кастомный градиентный бустинг
Фолд 1/5... R²: 0.8701, Время: 0.12с
Фолд 2/5... R²: 0.8959, Время: 0.11с
Фолд 3/5... R²: 0.9100, Время: 0.10с
Фолд 4/5... R²: 0.8751, Время: 0.08с
Фолд 5/5... R²: 0.9065, Время: 0.09с

Кросс-валидация:
R²: 0.8915 ± 0.0162
Время: 0.10 ± 0.01 секунд


# 6. Сравнить результаты с эталонной реализацией из библиотеки [scikit-learn](https://scikit-learn.org/stable/):
   * точность модели;
   * время обучения.

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [13]:
print("Sklearn GB")

# Создание и обучение модели sklearn
sklearn_gb = GradientBoostingRegressor(
    n_estimators=50, learning_rate=0.1, max_depth=4, random_state=42
)

# Замер времени и кросс-валидация
cv_start_time = time.time()
cv_results = cross_validate(
    sklearn_gb, X_train, y_train, cv=5, 
    scoring=['r2', 'neg_mean_squared_error'], 
    return_train_score=True
)
cv_time = time.time() - cv_start_time

# Результаты
test_r2 = cv_results['test_r2']
print(f"Кросс-валидация sklearn:")
print(f"R²: {test_r2.mean():.4f} ± {test_r2.std():.4f}")
print(f"Время: {cv_time:.2f} секунд")

Sklearn GB
Кросс-валидация sklearn:
R²: 0.8799 ± 0.0099
Время: 0.41 секунд


Отчет приложен в README.md